In [56]:
%pylab inline
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
from itertools import product


import seaborn
from sklearn import preprocessing
from matplotlib.colors import LogNorm
from sklearn.metrics import accuracy_score
from sklearn  import metrics

from sklearn.preprocessing import MinMaxScaler

Populating the interactive namespace from numpy and matplotlib


/Users/aurinko/anaconda/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['product']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [57]:
weth = pd.read_csv('WeatherSF.csv',',',  dayfirst=True)#, index_col=['Dates'], parse_dates=['Dates'])
#df.columns
weth.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,PRCP,PRCP_ATTRIBUTES,SNOW,SNOW_ATTRIBUTES,...,TAVG,TAVG_ATTRIBUTES,TMAX,TMAX_ATTRIBUTES,TMIN,TMIN_ATTRIBUTES,TOBS,TOBS_ATTRIBUTES,WT11,WT11_ATTRIBUTES
0,USW00023272,"SAN FRANCISCO DOWNTOWN, CA US",37.7705,-122.4269,45.7,2003-01-01,0.0,",,0,2400",NaN,NaN,...,NaN,NaN,59,",,0",46,",,0",NaN,NaN,NaN,NaN
1,USW00023272,"SAN FRANCISCO DOWNTOWN, CA US",37.7705,-122.4269,45.7,2003-01-02,0.0,",,0,2400",NaN,NaN,...,NaN,NaN,57,",,0",48,",,0",NaN,NaN,NaN,NaN
2,USW00023272,"SAN FRANCISCO DOWNTOWN, CA US",37.7705,-122.4269,45.7,2003-01-03,0.0,",,0,2400",NaN,NaN,...,NaN,NaN,63,",,0",47,",,0",NaN,NaN,NaN,NaN
3,USW00023272,"SAN FRANCISCO DOWNTOWN, CA US",37.7705,-122.4269,45.7,2003-01-04,0.0,",,0,2400",NaN,NaN,...,NaN,NaN,66,",,0",50,",,0",NaN,NaN,NaN,NaN
4,USW00023272,"SAN FRANCISCO DOWNTOWN, CA US",37.7705,-122.4269,45.7,2003-01-05,0.0,",,0,2400",NaN,NaN,...,NaN,NaN,69,",,0",49,",,0",NaN,NaN,NaN,NaN


In [58]:
weth.shape

(4627, 22)

In [59]:
weth.WT11_ATTRIBUTES.value_counts()

,,7    1
Name: WT11_ATTRIBUTES, dtype: int64

In [64]:
weth = weth.drop('NAME', axis=1)
weth = weth.drop('LATITUDE', axis=1)
weth = weth.drop('LONGITUDE', axis=1)
weth = weth.drop('ELEVATION', axis=1)
weth = weth.drop('TAVG', axis=1)
weth = weth.drop('TAVG_ATTRIBUTES', axis=1)
weth = weth.drop('STATION', axis=1)
weth = weth.drop('PRCP_ATTRIBUTES', axis=1)
weth = weth.drop('TMAX_ATTRIBUTES', axis=1)
weth = weth.drop('SNOW_ATTRIBUTES', axis=1)
weth = weth.drop('SNWD', axis=1)
weth = weth.drop('SNWD_ATTRIBUTES', axis=1)
weth = weth.drop('TOBS', axis=1)
weth = weth.drop('TOBS_ATTRIBUTES', axis=1)
weth = weth.drop('WT11_ATTRIBUTES', axis=1)
weth = weth.drop('TMIN_ATTRIBUTES', axis=1)
weth = weth.drop('WT11', axis=1)
weth

,DATE,PRCP,SNOW,TMAX,TMIN
0,2003-01-01,0.00,NaN,59,46
1,2003-01-02,0.00,NaN,57,48
2,2003-01-03,0.00,NaN,63,47
3,2003-01-04,0.00,NaN,66,50
4,2003-01-05,0.00,NaN,69,49
5,2003-01-06,0.00,NaN,69,52
6,2003-01-07,0.00,NaN,66,47
7,2003-01-08,0.00,NaN,63,45
8,2003-01-09,0.99,NaN,56,50
9,2003-01-10,0.13,NaN,61,53


In [73]:
weth.TMIN.isnull().sum()

0

In [75]:
weth.SNOW.isnull().sum()

0

In [74]:
weth.SNOW.fillna(1.0, inplace=True)

In [70]:
weth.shape

(4627, 5)

In [65]:
weth.SNOW.value_counts()

0.0    1377
Name: SNOW, dtype: int64

In [63]:
weth.TMIN_ATTRIBUTES.value_counts()

,,0     2917
,,W      890
,,X      809
,,Z        9
,O,0       1
,I,0       1
Name: TMIN_ATTRIBUTES, dtype: int64

In [76]:
weth

,DATE,PRCP,SNOW,TMAX,TMIN
0,2003-01-01,0.00,1.0,59,46
1,2003-01-02,0.00,1.0,57,48
2,2003-01-03,0.00,1.0,63,47
3,2003-01-04,0.00,1.0,66,50
4,2003-01-05,0.00,1.0,69,49
5,2003-01-06,0.00,1.0,69,52
6,2003-01-07,0.00,1.0,66,47
7,2003-01-08,0.00,1.0,63,45
8,2003-01-09,0.99,1.0,56,50
9,2003-01-10,0.13,1.0,61,53


In [78]:
weth.columns

Index([u'DATE', u'PRCP', u'SNOW', u'TMAX', u'TMIN'], dtype='object')

In [77]:
weth.to_csv('weather.csv', sep=',', index=False)